# Extracting time-series features from resting-state fMRI data

In [1]:
import os
import numpy as np
import nibabel as nib
import pandas as pd
from pyarrow import feather
from nilearn.image import math_img, resample_img, index_img, threshold_img
from matplotlib import pyplot as plt
from scipy import stats

%load_ext rpy2.ipython

In [2]:
%%R
# Load R packages
suppressPackageStartupMessages({
    library(tidyverse)
    library(FactoMineR)
    library(factoextra)
})

## Loading in the data

We will start by computing the mean framewise displacement (FD) for each participant so that we know which participant(s) to exclude from further analysis.
FD is computed using the method from [Power et al. (2012)](https://doi.org/10.1016/j.neuroimage.2011.10.018) and we apply the 'lenient' threshold described in [Parkes et al. (2018)](https://doi.org/10.1016/j.neuroimage.2017.12.073).

In [2]:
%%bash

# Define the path to the data
github_path=$(echo $(pwd) | tr -d ' ')

# Run the calc_FD.m script -- note that you might need to update your matlab path here
cd code/movement_analysis

# UCLA CNP
matlab -nodisplay -singleCompThread -r "calc_FD UCLA_CNP $github_path $github_path/data/movement_data/UCLA_CNP/; exit"

# ABIDE
matlab -nodisplay -singleCompThread -r "calc_FD ABIDE $github_path $github_path/data/movement_data/ABIDE/; exit"



                            < M A T L A B (R) >
                  Copyright 1984-2023 The MathWorks, Inc.
                  R2023a (9.14.0.2206163) 64-bit (maci64)
                             February 22, 2023

 
To get started, type doc.
For product information, visit www.mathworks.com.
 

                            < M A T L A B (R) >
                  Copyright 1984-2023 The MathWorks, Inc.
                  R2023a (9.14.0.2206163) 64-bit (maci64)
                             February 22, 2023

 
To get started, type doc.
For product information, visit www.mathworks.com.
 



FD data should be organized in a `.txt` file with two columns (comma-delimited): one for the `Sample_ID` and one for the `Mean_FD_Power`.


In [2]:
UCLA_CNP_mean_FD = pd.read_table('data/movement_data/UCLA_CNP/UCLA_CNP_Mean_FD_Power.txt', delimiter=',', header=None).rename(columns={0: 'Sample_ID', 1: 'Mean_FD_Power'})
ABIDE_mean_FD = pd.read_table('data/movement_data/ABIDE/ABIDE_Mean_FD_Power.txt', delimiter=',', header=None).rename(columns={0: 'Sample_ID', 1: 'Mean_FD_Power'})

# Print the first five rows of the UCLA CNP Mean FD
UCLA_CNP_mean_FD.head()

,Sample_ID,Mean_FD_Power
0,sub-10159,0.110490
1,sub-10171,0.214750
2,sub-10189,0.204260
3,sub-10206,0.064667
4,sub-10217,0.062641


We will drop any participants with a mean FD > 0.55mm per the 'lenient' threshold criteria and save the list of participants to drop to a .txt file:

In [3]:
# Identify any participants with Mean_FD_Power > 0.55 and write their sample IDs to a text file
UCLA_CNP_participants_to_drop = UCLA_CNP_mean_FD[UCLA_CNP_mean_FD['Mean_FD_Power'] > 0.55]['Sample_ID']
UCLA_CNP_participants_to_drop.to_csv('data/input_data/UCLA_CNP_participants_to_drop_lenient.txt', index=False, header=False)

ABIDE_participants_to_drop = ABIDE_mean_FD[ABIDE_mean_FD['Mean_FD_Power'] > 0.55]['Sample_ID']
ABIDE_participants_to_drop.to_csv('data/input_data/ABIDE_participants_to_drop_lenient.txt', index=False, header=False)

UCLA_CNP_participants_to_drop = UCLA_CNP_participants_to_drop.tolist()
ABIDE_participants_to_drop = ABIDE_participants_to_drop.tolist()
ABIDE_participants_to_drop = [str(x) for x in ABIDE_participants_to_drop]

# Print the number of participants to drop per study
print('UCLA CNP participants to drop: {}'.format(len(UCLA_CNP_participants_to_drop)))
print('ABIDE participants to drop: {}'.format(len(ABIDE_participants_to_drop)))

UCLA CNP participants to drop: 3
ABIDE participants to drop: 59



We will start with our resting-state fMRI data stored in a [`.feather` file](https://arrow.apache.org/docs/python/feather.html) (for easy conversion between R and Python).
Data should be organized in a long format, such that there is one row for each brain region and timepoint per participant.

In [4]:
# Define input time-series feather files for the two datasets
UCLA_CNP_input_time_series_data = pd.read_feather('data/input_data/UCLA_CNP_fMRI_TS.feather')
ABIDE_input_time_series_data = pd.read_feather('data/input_data/ABIDE_fMRI_TS.feather')

In [3]:
# Load information about the time-series features
univariate_TS_feature_info = pd.read_csv("data/feature_info/univariate_feature_info.csv")
pairwise_TS_feature_info = pd.read_csv("data/feature_info/pairwise_feature_info.csv")

Let's print out the first five rows of this time-series dataset for the UCLA CNP cohort:

In [5]:
UCLA_CNP_input_time_series_data.head()

,Sample_ID,Noise_Proc,Brain_Region,timepoint,values
0,sub-10159,AROMA+2P+GMR,ctx-lh-bankssts,1,4.071963
1,sub-10159,AROMA+2P+GMR,ctx-lh-caudalanteriorcingulate,1,1.078613
2,sub-10159,AROMA+2P+GMR,ctx-lh-caudalmiddlefrontal,1,-0.191693
3,sub-10159,AROMA+2P+GMR,ctx-lh-cuneus,1,0.888717
4,sub-10159,AROMA+2P+GMR,ctx-lh-entorhinal,1,-4.071331


The data should be structured such that there are five columns:
1. `Sample_ID`: The unique ID mapping to an individual participant.
2. `Noise_Proc`: Name of the noise processing procedure, useful for when multiple noise processing pipelines are evaluated.
3. `Brain_Region`: The name of the brain region.
4. `timepoint`: The timepoint corresponding to the BOLD frame.
5. `values`: The BOLD signal amplitude at the given timepoint.

## Extracting univariate time-series features

First, we will extract 25 univariate time-series features comprising the [`catch22`](https://doi.org/10.1007/s10618-019-00647-x) feature set, mean, standard deviation, and fractional amplitude of low-frequency fluctuations (fALFF).
The `catch22` features, mean, and SD can all be computed in R using the [`theft`](https://cran.r-project.org/web/packages/theft/vignettes/theft.html) package (collectively referred to as the `catch24` feature set), while the fALFF will be computed in Matlab.
Computing the `catch24` features will take several minutes, so feel free to hit play on the next code chunk and grab a coffee ☕️
(Alternatively, you can run this on a high-performance computing cluster if you prefer.)

In [8]:
%%R -i UCLA_CNP_input_time_series_data,ABIDE_input_time_series_data -o UCLA_CNP_catch24_features,ABIDE_catch24_features
# Load the theft and tidyr packages
library(theft)
library(tidyr)

# We can define a helper function to compute the `catch24` time-series features using the `theft` package
catch24_all_samples <- function(full_TS_data,
                                output_column_names = c("Output"),
                                unique_columns = c("Sample_ID", "Brain_Region", "Noise_Proc")) {
  
  
  # Merge columns into unique ID
  full_TS_data <- full_TS_data %>%
    tidyr::unite("Unique_ID", unique_columns, sep="__")
  
  # Compute the set of 24 time-series features using theft
  TS_catch24 <- theft::calculate_features(data = full_TS_data, 
                                          id_var = "Unique_ID", 
                                          time_var = "timepoint", 
                                          values_var = "values", 
                                          feature_set = "catch22",
                                          catch24 = TRUE)[[1]] %>%
    tidyr::separate("id", c(output_column_names), sep="__")

  # Return the resulting set of 24 features computed per brain region
  return(TS_catch24)
    
}

# Compute the 24 time-series features for UCLA CNP and ABIDE time-series data
UCLA_CNP_catch24_features <- catch24_all_samples(UCLA_CNP_input_time_series_data,
                                                 output_column_names = c("Sample_ID", "Brain_Region", "Noise_Proc"))
ABIDE_catch24_features <- catch24_all_samples(ABIDE_input_time_series_data,
                                                output_column_names = c("Sample_ID", "Brain_Region", "Noise_Proc"))
                                  


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
No IDs removed. All value vectors good for feature extraction.
Running computations for catch22...

Calculations completed for catch22.
No IDs removed. All value vectors good for feature extraction.
Running computations for catch22...

Calculations completed for catch22.
In addition: Warning messages:
1: Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(unique_columns)

  # Now:
  data %>% select(all_of(unique_columns))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was
generated. 
2: There was 1 warning in `dplyr::summarise()`.
ℹ In argument: `Rcatch22::catch22_all(.data$values, catch24 = catch24)`.
ℹ In group 1: `id = "sub-10159__Left-Accumbens-area__A

We will separately compute the fractional amplitude of low-frequency fluctuations (fALFF) using the Matlab script `compute_regional_fALFF.m` as follows (note: Matlab license is required to run this):

In [4]:
# Load in catch24 data once it's been created
UCLA_CNP_catch24_features = pd.read_feather('data/time_series_features/UCLA_CNP_catch24_features.feather')
ABIDE_catch24_features = pd.read_feather('data/time_series_features/ABIDE_catch24_features.feather')

In [7]:
%%bash 

# First, we need to convert our time-series feather file to a Matlab .mat file to be read in properly
UCLA_CNP_time_series_file_base='data/input_data/UCLA_CNP_fMRI_TS'
ABIDE_time_series_file_base='data/input_data/ABIDE_ASD_fMRI_TS'

# Run the feather_to_mat.py script with the file base as the input argument, indicating that the output file should be a mat file
python3 code/feature_extraction/feather_to_mat.py ${UCLA_CNP_time_series_file_base} mat
python3 code/feature_extraction/feather_to_mat.py ${ABIDE_time_series_file_base} mat

In [8]:
%%bash 

# Run the feather_to_mat.py script with the file base as the input argument, indicating that the output file should be a mat file
python3 code/feature_extraction/feather_to_mat.py data/input_data/UCLA_CNP_fMRI_TS mat
python3 code/feature_extraction/feather_to_mat.py data/input_data/ABIDE_fMRI_TS mat

# Define the path to the data
data_path=$(echo $(pwd) | tr -d ' ')

# Run the compute_regional_fALFF.m script -- note that you might need to update your matlab path here
cd code/feature_extraction

# UCLA CNP
TS_mat_file="$data_path/data/input_data/UCLA_CNP_fMRI_TS.mat"
output_mat_file="data/time_series_features/UCLA_CNP_fALFF.mat"
/Applications/MATLAB_R2023a.app/bin/matlab -nodisplay -singleCompThread -r "compute_regional_fALFF $data_path $TS_mat_file $output_mat_file; exit"

# ABIDE
TS_mat_file="data/input_data/ABIDE_fMRI_TS.mat"
output_mat_file="data/time_series_features/ABIDE_fALFF.mat"
/Applications/MATLAB_R2023a.app/bin/matlab -nodisplay -singleCompThread -r "compute_regional_fALFF $data_path $TS_mat_file $output_mat_file; exit"

# Convert the mat file back to feather for fALFF
python3 feather_to_mat.py ${data_path}/data/time_series_features/UCLA_CNP_fALFF feather
python3 feather_to_mat.py ${data_path}/data/time_series_features/ABIDE_fALFF feather



                            < M A T L A B (R) >
                  Copyright 1984-2023 The MathWorks, Inc.
                  R2023a (9.14.0.2206163) 64-bit (maci64)
                             February 22, 2023

 
To get started, type doc.
For product information, visit www.mathworks.com.
 

                            < M A T L A B (R) >
                  Copyright 1984-2023 The MathWorks, Inc.
                  R2023a (9.14.0.2206163) 64-bit (maci64)
                             February 22, 2023

 
To get started, type doc.
For product information, visit www.mathworks.com.
 


In [5]:
# Read in the fALFF feather files
UCLA_CNP_fALFF = pd.read_feather('data/time_series_features/UCLA_CNP_fALFF.feather')
ABIDE_fALFF = pd.read_feather('data/time_series_features/ABIDE_fALFF.feather')

# Remove whitespace from Brain_Region column in the fALFF dataframes
UCLA_CNP_fALFF['Brain_Region'] = UCLA_CNP_fALFF['Brain_Region'].str.strip()
ABIDE_fALFF['Brain_Region'] = ABIDE_fALFF['Brain_Region'].str.strip()

# Read in metadata files
UCLA_CNP_metadata = pd.read_feather('data/input_data/UCLA_CNP_sample_metadata.feather')
ABIDE_metadata = pd.read_feather('data/input_data/ABIDE_sample_metadata.feather')

In [ ]:
%%R -i UCLA_CNP_fALFF,ABIDE_fALFF,UCLA_CNP_catch24_features,ABIDE_catch24_features,UCLA_CNP_metadata,ABIDE_metadata,UCLA_CNP_participants_to_drop,ABIDE_participants_to_drop -o UCLA_CNP_catch25_filtered,ABIDE_catch25_filtered

source("code/feature_extraction/QC_functions_univariate.R")
univariate_feature_set <- "catch25"

UCLA_CNP_catch25_filtered <- run_QC_for_univariate_dataset(sample_metadata = UCLA_CNP_metadata,
                                                           univariate_feature_set = univariate_feature_set,
                                                           catch24_results = UCLA_CNP_catch24_features,
                                                           fALFF_results = UCLA_CNP_fALFF,
                                                           participants_to_drop = UCLA_CNP_participants_to_drop)


ABIDE_catch25_filtered <- run_QC_for_univariate_dataset(sample_metadata = ABIDE_metadata,
                                                       univariate_feature_set = univariate_feature_set,
                                                       catch24_results = ABIDE_catch24_features,
                                                       fALFF_results = ABIDE_fALFF,
                                                       participants_to_drop = ABIDE_participants_to_drop)




In [8]:
# Save the filtered data to feather files
UCLA_CNP_catch25_filtered.reset_index().to_feather('data/time_series_features/UCLA_CNP_catch25_filtered.feather')
ABIDE_catch25_filtered.reset_index().to_feather('data/time_series_features/ABIDE_catch25_filtered.feather')

# Save filtered metadata
UCLA_CNP_metadata_filtered = UCLA_CNP_metadata[UCLA_CNP_metadata['Sample_ID'].isin(UCLA_CNP_catch25_filtered['Sample_ID'])]
ABIDE_metadata_filtered = ABIDE_metadata[ABIDE_metadata['Sample_ID'].isin(ABIDE_catch25_filtered['Sample_ID'])]

UCLA_CNP_metadata_filtered.reset_index().to_feather('data/input_data/UCLA_CNP_sample_metadata_filtered.feather')
ABIDE_metadata_filtered.reset_index().to_feather('data/input_data/ABIDE_sample_metadata_filtered.feather')

## Extracting statistics of pairwise interactions (SPIs) for coupling strengths

For `pyspi` computations, we opted to run a distributed version on a high-performance computing cluster.
We parallelized computations by separating the fMRI time-series data into individualized `numpy` array files (`.npy`) to be parsed by individual HPC job nodes.

In [12]:
# Function to iterate over each Sample_ID in the given time series dataframe and save the corresponding time series to a numpy file
def split_MTS_into_npy(time_series_data, study):

    # Create output directory if it doesn't already exist
    try:
        os.makedirs("data/time_series_features/" + study, 
                    exist_ok = True)
    except:
        pass
        
    # Iterate over each participant
    for sample_id in time_series_data['Sample_ID'].unique().tolist():
       # Extract just that participant's data
        participant_TS_data = time_series_data.query('Sample_ID == @sample_id')

        # Reshape data from long to wide
        participant_TS_data_wide = participant_TS_data.pivot(index='Brain_Region', columns='timepoint', values='values').reset_index()

        # Convert to numpy array
        participant_TS_data_np = participant_TS_data_wide.drop(['Brain_Region'], axis=1).to_numpy()

        # Z-score each row
        data_norm = np.apply_along_axis(stats.zscore, 1, participant_TS_data_np)

        # Save the numpy array to a numpy binary file
        np.save('data/time_series_features/' + study + '/' + sample_id + '.npy', data_norm)

# Call the function for UCLA_CNP and ABIDE
split_MTS_into_npy(UCLA_CNP_input_time_series_data, 'UCLA_CNP')
split_MTS_into_npy(ABIDE_input_time_series_data, 'ABIDE')


Next, we need to create a `.yaml` file for each study, containing information about the file location for each sample ID, along with a label containing diagnostic group.
We specify that the dimension order is presented with brain regions ('processes') as rows and timepoints as columns by setting `dim_order` to `ps`.


In [13]:
%%bash

ID_var='Sample_ID'
label_vars='Diagnosis'

# Run for UCLA CNP
data_dir='data/time_series_features/UCLA_CNP/'
sample_metadata_file='data/input_data/UCLA_CNP_sample_metadata.feather'

Rscript code/feature_extraction/create_yaml_for_samples.R  \
    --data_dir $data_dir \
    --sample_metadata_file $sample_metadata_file \
    --ID_var $ID_var \
    --label_vars $label_vars \
    --dim_order ps

# Run for ABIDE
data_dir='data/time_series_features/ABIDE/'
sample_metadata_file='data/input_data/ABIDE_sample_metadata.feather'

Rscript code/feature_extraction/create_yaml_for_samples.R  \
    --data_dir $data_dir \
    --sample_metadata_file $sample_metadata_file \
    --ID_var $ID_var \
    --label_vars $label_vars \
    --dim_order ps

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors



YAML output: data/time_series_features/UCLA_CNP/sample.yaml 


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors



YAML output: data/time_series_features/ABIDE/sample.yaml 


We recommend that you clone the [pyspi-distribute](https://github.com/olivercliff/pyspi-distribute) repository to make use of job distribution on an HPC cluster.
To do so, you can clone with the following command:
```
git clone https://github.com/olivercliff/pyspi-distribute.git
```

In [ ]:
%%bash

pyspi_distribute_path='../pyspi-distribute/' # Change to the path where you cloned the pyspi-distribute repo
conda_env='base' # Change to the name of the conda environment you want to use
queue='normal' # Change to the name of the queue you want to use for your HPC
pyspi_walltime_hrs=2 # Change to the maximum walltime in hours you want to allow for each job
pyspi_ncpus=1 # Change to the number of CPUs you want to use for each job
pyspi_mem=20 # Change to the amount of memory in GB you want to use for each job

# UCLA CNP
python3 ${pyspi_distribute_path}/distribute_jobs.py \
--data_dir data/time_series_features/UCLA_CNP/ \
--calc_file_name calc.pkl \
--compute_file ${pyspi_distribute_path}/pyspi_compute.py \
--template_pbs_file code/feature_extraction/template_pyspi_distribute.pbs \
--pyspi_config code/feature_extraction/SPIs_14_config.yaml \
--sample_yaml data/time_series_features/UCLA_CNP/sample.yaml \
--pbs_notify a \
--email [your_email_address_here] \
--conda_env $conda_env \
--queue $queue \
--walltime_hrs $pyspi_walltime_hrs \
--cpu $pyspi_ncpus \
--mem $pyspi_mem \
--table_only

# ABIDE
python3 ${pyspi_distribute_path}/distribute_jobs.py \
--data_dir data/time_series_features/ABIDE/ \
--calc_file_name calc.pkl \
--compute_file ${pyspi_distribute_path}/pyspi_compute.py \
--template_pbs_file code/feature_extraction/template_pyspi_distribute.pbs \
--pyspi_config code/feature_extraction/SPIs_14_config.yaml \
--sample_yaml data/time_series_features/ABIDE/sample.yaml \
--pbs_notify a \
--email [your_email_address_here] \
--conda_env $conda_env \
--queue $queue \
--walltime_hrs $pyspi_walltime_hrs \
--cpu $pyspi_ncpus \
--mem $pyspi_mem \
--table_only

After running this for all participants, you should have a folder for each participant under `data/time_series_features/[Study]/[Sample_ID]` containing the following files:
- `pyspi_run.pbs`: This is the job submission script that was automatically created and submitted for this participant.
- `pyspi_run.out`: Notes where the `.npy` file was loaded from and where the resulting `calc.pkl` file was saved for this participant.
- `pbsjob.out`: All outputs printed to `stdout`, and `stderr` if applicable. Useful for debugging/confirming that everything ran okay.
- `calc.pkl`: pyspi computation results for this participant, saved as a `pandas DataFrame`.

In [15]:
example_calc_table = pd.read_pickle('data/time_series_features/UCLA_CNP/sub-70086/calc.pkl')
example_calc_table.head()

,brain_region_from,value,SPI,brain_region_to
0,proc-0,NaN,cov_EmpiricalCovariance,proc-0
1,proc-1,-0.043263,cov_EmpiricalCovariance,proc-0
2,proc-2,-0.103124,cov_EmpiricalCovariance,proc-0
3,proc-3,0.041604,cov_EmpiricalCovariance,proc-0
4,proc-4,-0.097217,cov_EmpiricalCovariance,proc-0


As shown in the above example table, the results are stored in a long format such that there is a `brain_region_from` and a `brain_region_to`, an `SPI` (in this case, `cov_EmpiricalCovariance` which is equivalent to the Pearson correlation coefficient) and a `value`.
Note that the `value` is `NaN` in the first row since that corresponds to a self-connection, which is not a valid measurement (and will subsequently be dropped when we merge the results).
The default output value nomenclature for `pyspi` is process IDs, such as `proc-0`, `proc-1`, etc., which we will map to brain region names subsequently, too.

Merging and data cleaning is accomplished with the script `code/feature_extraction/merge_pyspi_data.py`, which can be run as follows:

In [ ]:
%%bash

python3 code/feature_extraction/merge_pyspi_data.py \
--data_path data/ \
--dataset_ID UCLA_CNP \
--pkl_file calc.pkl \
--pairwise_feature_set pyspi14 \
--brain_region_lookup data/input_data/UCLA_CNP_Brain_Region_Lookup.feather

At this point, you should have the following files in your `data/time_series_features/` path for subsequent classification analysis:
- `ABIDE_catch25_filtered.feather`
- `ABIDE_pyspi14_filtered.feather`
- `UCLA_CNP_catch25_filtered.feather`
- `UCLA_CNP_pyspi14_filtered.feather`


Our last step will be to compare across the univariate and pairwise feature sets for each dataset and find the participants with data available for both feature sets as a quality control check:

In [10]:
def intersection_univariate_pairwise(data_path, dataset_ID, univariate_feature_set, pairwise_feature_set):

    # Load in data on samples with univariate feature data
    filtered_univariate_data = pd.read_feather(f"{data_path}/{dataset_ID}_{univariate_feature_set}_filtered.feather")
    filtered_univariate_samples = pd.DataFrame(filtered_univariate_data.Sample_ID.unique(), columns=["Sample_ID"])
    
    # Load in data on samples with pairwise feature data
    filtered_pairwise_data = pd.read_feather(f"{data_path}/{dataset_ID}_{pairwise_feature_set}_filtered.feather")
    filtered_pairwise_samples = pd.DataFrame(filtered_pairwise_data.Sample_ID.unique(), columns=["Sample_ID"])
    
    # Merge the two datasets
    merged_sample_info = pd.merge(filtered_univariate_samples, filtered_pairwise_samples, how="inner")
    merged_sample_info.to_feather(f"{data_path}/{dataset_ID}_filtered_sample_info_{univariate_feature_set}_{pairwise_feature_set}.feather")
    
# Run for UCLA CNP
intersection_univariate_pairwise(data_path='data/time_series_features',
                                 dataset_ID='UCLA_CNP',
                                 univariate_feature_set='catch25',
                                 pairwise_feature_set='pyspi14')

# Run for ABIDE
intersection_univariate_pairwise(data_path='data/time_series_features',
                                 dataset_ID='ABIDE',
                                 univariate_feature_set='catch25',
                                 pairwise_feature_set='pyspi14')

# Apply this final filter to the .feather files
UCLA_CNP_filtered_sample_info = pd.read_feather('data/time_series_features/UCLA_CNP_filtered_sample_info_catch25_pyspi14.feather')

UCLA_CNP_catch25_filtered = pd.read_feather('data/time_series_features/UCLA_CNP_catch25_filtered.feather').query("Sample_ID in @UCLA_CNP_filtered_sample_info.Sample_ID.unique().tolist()")
UCLA_CNP_catch25_filtered.to_feather('data/time_series_features/UCLA_CNP_catch25_filtered.feather')

UCLA_CNP_pyspi14_filtered = pd.read_feather('data/time_series_features/UCLA_CNP_pyspi14_filtered.feather').query("Sample_ID in @UCLA_CNP_filtered_sample_info.Sample_ID.unique().tolist()")
UCLA_CNP_pyspi14_filtered.to_feather('data/time_series_features/UCLA_CNP_pyspi14_filtered.feather')

ABIDE_filtered_sample_info = pd.read_feather('data/time_series_features/ABIDE_filtered_sample_info_catch25_pyspi14.feather')

ABIDE_catch25_filtered = pd.read_feather('data/time_series_features/ABIDE_catch25_filtered.feather').query("Sample_ID in @ABIDE_filtered_sample_info.Sample_ID.unique().tolist()")
ABIDE_catch25_filtered.reset_index().to_feather('data/time_series_features/ABIDE_catch25_filtered.feather')

ABIDE_pyspi14_filtered = pd.read_feather('data/time_series_features/ABIDE_pyspi14_filtered.feather').query("Sample_ID in @ABIDE_filtered_sample_info.Sample_ID.unique().tolist()")
ABIDE_pyspi14_filtered.reset_index().to_feather('data/time_series_features/ABIDE_pyspi14_filtered.feather')

Lastly, we will fit a principal components analysis (PCA) on the univariate region times feature combination matrices to retain the first 25 PCs per disorder, as a validation analysis for dimensionality reduction.

In [24]:
%%R -i UCLA_CNP_metadata,ABIDE_metadata -o first_25_PCs_by_disorder

# Helper function to compute n_PCs from a given input dataset
compute_first_n_PCs <- function(univariate_feature_data, dataset_ID, metadata, disorder, n_PCs) {
  
  # Extract the univariate region times feature data for the given disorder
  data_for_PCA <- univariate_feature_data %>%
    left_join(., metadata, by = join_by(Sample_ID)) %>%
    filter(Diagnosis %in% c("Control", disorder)) %>%
    mutate(unique_ID = paste0(names, "__", Brain_Region), .keep="unused") %>%
    dplyr::select(unique_ID, Sample_ID, Diagnosis, values) %>%
    pivot_wider(id_cols=c(Sample_ID, Diagnosis), names_from=unique_ID, values_from=values) %>%
    mutate(Diagnosis=factor(Diagnosis, levels=c("Control", disorder)))
  
  # Compute PCA
  pca_res <- PCA(select(data_for_PCA, c(-Sample_ID, -Diagnosis)), ncp=n_PCs, graph = FALSE, scale.unit = TRUE)
  pca_scores <- as.data.frame(pca_res$ind$coord) %>%
    mutate(Sample_ID = data_for_PCA$Sample_ID,
           Diagnosis = data_for_PCA$Diagnosis,
           Disorder = disorder,
           Study = dataset_ID)
  
  return(pca_scores)
}

# Iterate over each disorder and compute the first 25 PCs
first_25_PCs_by_disorder_list <- list()
for (disorder in c("Schizophrenia", "Bipolar", "ADHD")) {
  pca_scores_disorder <- compute_first_n_PCs(UCLA_CNP_catch25_filtered, 
                                             dataset_ID='UCLA_CNP', 
                                             metadata=UCLA_CNP_metadata, 
                                             disorder=disorder, 
                                             n_PCs=25)
  first_25_PCs_by_disorder_list[[disorder]] <- pca_scores_disorder
}
for (disorder in c("ASD")) {
  pca_scores_disorder <- compute_first_n_PCs(ABIDE_catch25_filtered, 
                                             dataset_ID='ABIDE', 
                                             metadata=ABIDE_metadata, 
                                             disorder=disorder, 
                                             n_PCs=25)
  first_25_PCs_by_disorder_list[[disorder]] <- pca_scores_disorder
}
first_25_PCs_by_disorder = do.call(rbind, first_25_PCs_by_disorder_list)

In addition: Warning message:
In PCA(select(data_for_PCA, c(-Sample_ID, -Diagnosis)), ncp = n_PCs,  :
  Missing values are imputed by the mean of the variable: you should use the imputePCA function of the missMDA package


In [25]:
# Save first_25_PCs_by_disorder to a feather file
first_25_PCs_by_disorder.to_feather('data/time_series_features/univariate_combo_first25_PCs.feather')